In [11]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [12]:
import os

In [13]:
os.listdir()


['1_notMNIST.ipynb',
 '2_fullyconnected.ipynb',
 '3_regularization.ipynb',
 'notMNIST.pickle',
 'notMNIST_clean.pickle',
 'notMNIST_large',
 'notMNIST_large.tar.gz',
 'notMNIST_small',
 'notMNIST_small.tar.gz']

In [14]:
os.chdir('E:\Manish\Machine Learning\Deep Learning  - Udacity\Python Assignments')
os.listdir()

['1_notMNIST.ipynb',
 '2_fullyconnected.ipynb',
 '3_regularization.ipynb',
 'notMNIST.pickle',
 'notMNIST_clean.pickle',
 'notMNIST_large',
 'notMNIST_large.tar.gz',
 'notMNIST_small',
 'notMNIST_small.tar.gz']

In [15]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [16]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [17]:
batch_size = 800
L2NormConst = 0.01
image_size = 28
num_labels = 10
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  
    # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + tf.add_n([tf.nn.l2_loss(weights)]) * L2NormConst
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [18]:

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [19]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 46.028225
Minibatch accuracy: 13.1%
Validation accuracy: 15.2%
Minibatch loss at step 500: 0.918375
Minibatch accuracy: 82.0%
Validation accuracy: 82.4%
Minibatch loss at step 1000: 0.718194
Minibatch accuracy: 84.0%
Validation accuracy: 82.5%
Minibatch loss at step 1500: 0.760242
Minibatch accuracy: 80.5%
Validation accuracy: 82.8%
Minibatch loss at step 2000: 0.691065
Minibatch accuracy: 83.2%
Validation accuracy: 82.7%
Minibatch loss at step 2500: 0.733406
Minibatch accuracy: 82.1%
Validation accuracy: 82.8%
Minibatch loss at step 3000: 0.757244
Minibatch accuracy: 81.4%
Validation accuracy: 82.7%
Minibatch loss at step 3500: 0.645852
Minibatch accuracy: 84.6%
Validation accuracy: 82.7%
Minibatch loss at step 4000: 0.684176
Minibatch accuracy: 83.1%
Validation accuracy: 82.6%
Minibatch loss at step 4500: 0.760616
Minibatch accuracy: 83.0%
Validation accuracy: 82.6%
Minibatch loss at step 5000: 0.690076
Minibatch accuracy: 83.2%
Validation accura

In [20]:
#Changes for Relu - 1 hidden layer - Neural Network
batch_size = 500
hidden_nodes = 1024
L2NormConst = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
   # Variables - intermediate
  weights_int = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_int = tf.Variable(tf.ones([hidden_nodes])/100)
    
   # Training computation - intermediate.
  logits_int = tf.matmul(tf_train_dataset, weights_int) + biases_int
  
  #print('Training set', tf_train_dataset.shape)
  #print('Logit In', logits_int.shape)  

    # Adding ReLu 
  int_output = tf.nn.relu(logits_int) 
  #print('ReLu output set', int_output.shape)  
   
    # Variables.
  weights = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
    
  # Training computation.
  logits = tf.matmul(int_output, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+ tf.add_n([tf.nn.l2_loss(weights)+tf.nn.l2_loss(weights_int)]) * L2NormConst

  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul((tf.nn.relu(tf.matmul(tf_valid_dataset,weights_int) + biases_int)), weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul((tf.nn.relu(tf.matmul(tf_test_dataset,weights_int) + biases_int)), weights) + biases)

In [21]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 616.146973
Minibatch accuracy: 9.6%
Validation accuracy: 36.8%
Minibatch loss at step 1000: 212.473297
Minibatch accuracy: 83.2%
Validation accuracy: 82.9%
Minibatch loss at step 2000: 139.897461
Minibatch accuracy: 85.0%
Validation accuracy: 83.7%
Minibatch loss at step 3000: 92.804535
Minibatch accuracy: 89.4%
Validation accuracy: 85.3%
Minibatch loss at step 4000: 62.212433
Minibatch accuracy: 87.2%
Validation accuracy: 85.4%
Minibatch loss at step 5000: 41.650185
Minibatch accuracy: 89.4%
Validation accuracy: 86.1%
Minibatch loss at step 6000: 27.984501
Minibatch accuracy: 90.4%
Validation accuracy: 87.2%
Minibatch loss at step 7000: 18.864307
Minibatch accuracy: 92.6%
Validation accuracy: 87.8%
Minibatch loss at step 8000: 12.736120
Minibatch accuracy: 91.6%
Validation accuracy: 88.5%
Minibatch loss at step 9000: 8.645971
Minibatch accuracy: 92.6%
Validation accuracy: 88.7%
Minibatch loss at step 10000: 5.878602
Minibatch accuracy: 94.2%
Valid

In [22]:
#Changes for Relu - 1 hidden layer - Neural Network

batch_size = 30 # reducing batch size to check overfitting
hidden_nodes = 1024
L2NormConst = 0

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
   # Variables - intermediate
  weights_int = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_int = tf.Variable(tf.ones([hidden_nodes])/100)
    
   # Training computation - intermediate.
  logits_int = tf.matmul(tf_train_dataset, weights_int) + biases_int
  
  #print('Training set', tf_train_dataset.shape)
  #print('Logit In', logits_int.shape)  

    # Adding ReLu 
  int_output = tf.nn.relu(logits_int) 
  #print('ReLu output set', int_output.shape)  
   
    # Variables.
  weights = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
    
  # Training computation.
  logits = tf.matmul(int_output, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+ tf.add_n([tf.nn.l2_loss(weights)+tf.nn.l2_loss(weights_int)]) * L2NormConst

  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul((tf.nn.relu(tf.matmul(tf_valid_dataset,weights_int) + biases_int)), weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul((tf.nn.relu(tf.matmul(tf_test_dataset,weights_int) + biases_int)), weights) + biases)

In [23]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 296.076813
Minibatch accuracy: 23.3%
Validation accuracy: 16.0%
Minibatch loss at step 1000: 11.678220
Minibatch accuracy: 83.3%
Validation accuracy: 79.1%
Minibatch loss at step 2000: 19.562374
Minibatch accuracy: 83.3%
Validation accuracy: 79.3%
Minibatch loss at step 3000: 4.109977
Minibatch accuracy: 76.7%
Validation accuracy: 77.6%
Minibatch loss at step 4000: 0.142419
Minibatch accuracy: 96.7%
Validation accuracy: 79.3%
Minibatch loss at step 5000: 5.982605
Minibatch accuracy: 76.7%
Validation accuracy: 78.7%
Minibatch loss at step 6000: 3.199952
Minibatch accuracy: 76.7%
Validation accuracy: 79.5%
Minibatch loss at step 7000: 2.732478
Minibatch accuracy: 86.7%
Validation accuracy: 79.2%
Minibatch loss at step 8000: 3.716715
Minibatch accuracy: 86.7%
Validation accuracy: 80.8%
Minibatch loss at step 9000: 9.428632
Minibatch accuracy: 66.7%
Validation accuracy: 81.5%
Minibatch loss at step 10000: 4.499637
Minibatch accuracy: 83.3%
Validation a

In [67]:
batch_size = 5
L2NormConst = 0
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  
    # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + tf.add_n([tf.nn.l2_loss(weights)]) * L2NormConst
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [68]:
num_steps = 1000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 29.286966
Minibatch accuracy: 20.0%
Validation accuracy: 7.0%
Minibatch loss at step 100: 4.513234
Minibatch accuracy: 60.0%
Validation accuracy: 60.1%
Minibatch loss at step 200: 1.037190
Minibatch accuracy: 80.0%
Validation accuracy: 68.0%
Minibatch loss at step 300: 3.189623
Minibatch accuracy: 60.0%
Validation accuracy: 69.7%
Minibatch loss at step 400: 4.948014
Minibatch accuracy: 60.0%
Validation accuracy: 69.2%
Minibatch loss at step 500: 2.341604
Minibatch accuracy: 60.0%
Validation accuracy: 72.9%
Minibatch loss at step 600: 0.269194
Minibatch accuracy: 80.0%
Validation accuracy: 70.7%
Minibatch loss at step 700: 4.631228
Minibatch accuracy: 80.0%
Validation accuracy: 73.3%
Minibatch loss at step 800: 1.693470
Minibatch accuracy: 80.0%
Validation accuracy: 73.5%
Minibatch loss at step 900: 1.268430
Minibatch accuracy: 80.0%
Validation accuracy: 73.2%
Test accuracy: 79.7%


In [43]:
#Changes for Relu - 3 hidden layer - Neural Network
batch_size = 400
hidden_nodes_1 = 1024
hidden_nodes_2 = 500
hidden_nodes_3 = 200
L2NormConst = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Random prob for dropout
  keep_prob = tf.constant(np.random.uniform(0,1),shape=())  
  
  # Adding Dropout - layer 1
  tf_train_dataset = tf.nn.dropout(tf_train_dataset, keep_prob)

   # Variables - layer 1
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_1],stddev=0.03))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes_1]))
    
   # Training computation - intermediate.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  
  #print('Training set', tf_train_dataset.shape)
  #print('Logit In', logits_int.shape)  

    # Adding ReLu 
  output_1 = tf.nn.relu(logits_1)  
   # Adding Dropout - layer 2
  #output_1 = tf.nn.dropout(output_1, keep_prob)

    
     # Variables - layer 2
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_1, hidden_nodes_2],stddev=0.01))
  biases_2 = tf.Variable(tf.zeros([hidden_nodes_2]))
    
   # Training computation - intermediate.
  logits_2 = tf.matmul(output_1, weights_2) + biases_2
  
  #print('Training set', tf_train_dataset.shape)
  #print('Logit In', logits_int.shape)  

    # Adding ReLu 
  output_2 = tf.nn.relu(logits_2) 
    
     # Adding Dropout - Output layer 
  #output_2 = tf.nn.dropout(output_2, keep_prob)


     # Variables - layer 2
  weights_3 = tf.Variable(
    tf.truncated_normal([hidden_nodes_2, hidden_nodes_3],stddev=0.01))
  biases_3 = tf.Variable(tf.zeros([hidden_nodes_3]))
    
   # Training computation - intermediate.
  logits_3 = tf.matmul(output_2, weights_3) + biases_3
  


    # Adding ReLu 
  output_3 = tf.nn.relu(logits_3) 
    
     # Adding Dropout - Output layer 
  #output_3 = tf.nn.dropout(output_3, keep_prob)
   
    # Variables.
  weights = tf.Variable(
    tf.truncated_normal([hidden_nodes_3, num_labels],stddev=0.01))
  biases = tf.Variable(tf.zeros([num_labels]))
  
    
  # Training computation.
  logits = tf.matmul(output_3, weights) + biases
  print('logits', logits.shape)
  print('tf_train_labels', tf_train_labels.shape)  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + tf.add_n([tf.nn.l2_loss(weights)+tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)+tf.nn.l2_loss(weights_3)]) * L2NormConst
  print('loss', loss.shape)  
  
  # Optimizer.
  global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.7,staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)  
  
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul((tf.nn.relu(tf.matmul((tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights_1)+biases_1),weights_2)+biases_2)),weights_3) + biases_3)), weights) + biases)
  test_prediction = tf.nn.softmax(
    tf.matmul((tf.nn.relu(tf.matmul((tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_1)+biases_1),weights_2)+biases_2)),weights_3) + biases_3)), weights) + biases)

logits (400, 10)
tf_train_labels (400, 10)
loss ()


In [44]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.605446
Minibatch accuracy: 7.5%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.655792
Minibatch accuracy: 86.5%
Validation accuracy: 86.4%
Minibatch loss at step 1000: 0.583695
Minibatch accuracy: 88.2%
Validation accuracy: 88.1%
Minibatch loss at step 1500: 0.506168
Minibatch accuracy: 89.8%
Validation accuracy: 88.8%
Minibatch loss at step 2000: 0.515519
Minibatch accuracy: 87.5%
Validation accuracy: 89.5%
Minibatch loss at step 2500: 0.374113
Minibatch accuracy: 92.5%
Validation accuracy: 89.6%
Minibatch loss at step 3000: 0.468667
Minibatch accuracy: 89.0%
Validation accuracy: 89.6%
Minibatch loss at step 3500: 0.400737
Minibatch accuracy: 91.2%
Validation accuracy: 89.9%
Minibatch loss at step 4000: 0.408790
Minibatch accuracy: 91.8%
Validation accuracy: 90.3%
Minibatch loss at step 4500: 0.364470
Minibatch accuracy: 91.8%
Validation accuracy: 90.3%
Minibatch loss at step 5000: 0.421242
Minibatch accuracy: 91.5%
Validation accuracy

In [184]:
np.random.uniform(0,1)

0.3391445489401571